In [1]:
import numpy as np

In [2]:
import numpy as np
A = np.array([[3, -1, 1], [2, -5, -3], [1, 1, -1]])
b = np.array([4, -17, 0])[:, np.newaxis]
A_extended = np.hstack((A, b))
print(f'rank A = {np.linalg.matrix_rank(A)}')
print(f'rank A_extended = {np.linalg.matrix_rank(A_extended)}')

rank A = 3
rank A_extended = 3


In [3]:
A = np.array([[2, -4, 6], [1, -2, 3], [3, -6, 9]])
b = np.array([1, -2, 5])[:, np.newaxis]
A_extended = np.hstack((A, b))
print(f'rank A = {np.linalg.matrix_rank(A)}')
print(f'rank A_extended = {np.linalg.matrix_rank(A_extended)}')

rank A = 1
rank A_extended = 2


In [4]:
A = np.array([[1, 2, 5], [3, 1, -8]])
b = np.array([4, -2])[:, np.newaxis]
A_extended = np.hstack((A, b))
print(f'rank A = {np.linalg.matrix_rank(A)}')
print(f'rank A_extended = {np.linalg.matrix_rank(A_extended)}')

rank A = 2
rank A_extended = 2


In [5]:
A = np.array([[1, 3, -2, 4], [0, 5, 0, 1], [0, 0, 3, 0], [0, 0, 0,
2]])
b = np.array([[3], [2], [4], [1]])
A_ext = np.hstack((A, b))
print(f'Система совместна: {np.linalg.matrix_rank(A) == np.linalg.matrix_rank(A_ext)}')
print(f'Имеет единственное решение: {np.linalg.matrix_rank(A) == np.linalg.matrix_rank(A_ext) == 4}')

Система совместна: True
Имеет единственное решение: True


In [6]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
A[2] = A[2] - 2*A[1] + A[0]
A

array([[1, 2, 3],
       [4, 5, 6],
       [0, 0, 0]])

In [7]:
def kramer(A, b):
    assert A.shape[0] == A.shape[1], 'Wrong matrix size!'
    det_A = np.linalg.det(A)
    result = []
    if np.abs(det_A) > 0.000001: #может быть ошибка округления
        for i in range(A.shape[0]):
            A_i = np.hstack((A[:, :i], b, A[:, i+1:]))
            result.append(np.linalg.det(A_i) / det_A)
        return np.round(result, 2)
    else:
        return 'det A is 0'

In [8]:
A = np.array([[1, -2], [3, -4]])
b = np.array([[1, 7]]).T
print(f'a) {kramer(A, b)}')

A = np.array([[2, -1, 5], [1, 1, -3], [2, 4, 1]])
b = np.array([[10, -2, 1]]).T
print(f'б) {kramer(A, b)}')

a) [5. 2.]
б) [ 2. -1.  1.]


In [9]:
def LU(A):
    assert A.shape[0] == A.shape[1] and A[0][0] !=0 and np.abs(np.linalg.det(A)) > 0.00001, 'Невозможно разложить матрицу'
    L = np.zeros_like(A)
    U = np.zeros_like(A)
    # первая строка U и первый столбец L
    U[0] = A[0]
    L[:, 0] = A[:, 0] / U[0, 0]
    for i in range(A.shape[1]):
        for j in range(i, A.shape[1]):
            U[0, i] = A[0, i]
            L[i, 0] = A[i, 0] / U[0, 0]
            u_sum, l_sum = 0, 0
            for k in range(i):
                u_sum += L[i, k] * U[k, j]
                l_sum += L[j, k] * U[k, i]
            U[i, j] = A[i, j] - u_sum
            L[j, i] = (A[j, i] - l_sum) / U[i, i]
    return U, L

In [10]:
# a)
A = np.array([[1, 2, 4], [2, 9, 12], [3, 26, 30]])
U, L = LU(A)
print("a) ")
print("Матрица U:")
print(U)
print("Матрица L:")
print(L)
print("Матрица LU == A:")
print(L.dot(U))
print(A)
# б)
A = np.array([[1, 1, 2, 4], [2, 5, 8, 9], [3, 18, 29, 18], [4, 22, 53,
33]])
U, L = LU(A)
print("б) ")
print("Матрица U:")
print(U)
print("Матрица L:")
print(L)
print("Матрица LU == A:")
print(L.dot(U))
print(A)

a) 
Матрица U:
[[1 2 4]
 [0 5 4]
 [0 0 2]]
Матрица L:
[[1 0 0]
 [2 1 0]
 [3 4 1]]
Матрица LU == A:
[[ 1  2  4]
 [ 2  9 12]
 [ 3 26 30]]
[[ 1  2  4]
 [ 2  9 12]
 [ 3 26 30]]
б) 
Матрица U:
[[1 1 2 4]
 [0 3 4 1]
 [0 0 3 1]
 [0 0 0 4]]
Матрица L:
[[1 0 0 0]
 [2 1 0 0]
 [3 5 1 0]
 [4 6 7 1]]
Матрица LU == A:
[[ 1  1  2  4]
 [ 2  5  8  9]
 [ 3 18 29 18]
 [ 4 22 53 33]]
[[ 1  1  2  4]
 [ 2  5  8  9]
 [ 3 18 29 18]
 [ 4 22 53 33]]


In [11]:
def LU_slau(A, b):
    U, L = LU(A)
    y = np.zeros(A.shape[0])
    x = np.zeros(A.shape[0])
    for i in range(A.shape[0]):
        y[i] = b[i] - (L[i, :i] * y[:i]).sum()
    for i in range(A.shape[0]-1, -1, -1):
        x[i] = (y[i] - (U[i, i+1:]*x[i+1:]).sum())/U[i, i]
    return x

In [12]:
A = np.array([[2, 1, 3], [11, 7, 5], [9, 8, 4]])
b = np.array([1, -6, 5])
x = LU_slau(A, b)
print(f'x = {x}')
print('Проверка')
for i in range(len(A)):
    print((x * A[0]).sum() == b[0])

x = [-4.41666667  4.08333333  1.91666667]
Проверка
True
True
True
